In [1]:
import cv2
import numpy as np
from collections import deque
import pickle
import dill

# lsFilename, outfile = ['video/190502_00-002.mp4'], 'cam00.txt'
lsFilename, outfile = ['video/190502_01-003.mp4'], 'cam01.txt'
# lsFilename, outfile = ['video/190502_02-004.mp4'], 'cam02.txt'
# lsFilename, outfile = ['video/190502_03-005.mp4'], 'cam03.txt'
# lsFilename, outfile = ['video/190502_04-007.mp4'], 'cam04.txt'
# lsFilename, outfile = ['video/190502_05-008.mp4'], 'cam05.txt'
# lsFilename, outfile = ['video/190502_06-009.mp4'], 'cam06.txt'
# lsFilename, outfile = ['video/190502_07-010.mp4'], 'cam07.txt'
# lsFilename, outfile = ['video/190502_08-011.mp4'], 'cam08.txt'
# lsFilename, outfile = ['video/190502_09-012.mp4'], 'cam09.txt'
# lsFilename, outfile = ['video/190502_10-014.mp4'], 'cam10.txt'
# lsFilename, outfile = ['video/190502_11-015.mp4'], 'cam11.txt'

nFrame4calib = 100
bShow = True
# bShow = False
bRandomSample = True
# bRandomSample = False
# delay = 150
delay = 10
# delay = 0

nCorners = (5,7)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((nCorners[0]*nCorners[1],3), np.float32)
objp[:,:2] = np.mgrid[0:nCorners[0],0:nCorners[1]].T.reshape(-1,2) * 0.0288

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
frames = []

valid_frames = []
valid_grays = []
valid_corners = []

In [2]:
nFrame = 0
for filename in lsFilename:
    cap = cv2.VideoCapture(filename)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")

    nFrame += int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print 'total :',nFrame

total : 541


In [3]:
n = 0
for filename in lsFilename:
    print filename
    cap = cv2.VideoCapture(filename)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
        continue

    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            find, corners = cv2.findChessboardCorners(gray, nCorners, None)
            n += 1
            print n,
            if find and (corners.reshape((-1,2)).max(0) - corners.reshape((-1,2)).min(0)).min() > 200:
                valid_frames.append(frame)
                valid_grays.append(gray)
                valid_corners.append(corners)
                print "find!", len(valid_frames),

            
        else: 
            break
    cap.release()

nFrame = len(valid_frames)
print 
print "Detected %d frames" % nFrame

filename = 'globalsave1.pkl'
dill.dump_session(filename)


video/190502_01-003.mp4
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 find! 1 81 find! 2 82 find! 3 83 find! 4 84 find! 5 85 find! 6 86 find! 7 87 find! 8 88 find! 9 89 find! 10 90 find! 11 91 find! 12 92 find! 13 93 find! 14 94 find! 15 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 find! 16 127 find! 17 128 find! 18 129 find! 19 130 find! 20 131 find! 21 132 find! 22 133 find! 23 134 find! 24 135 find! 25 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 find! 26 156 find! 27 157 find! 28 158 find! 29 159 find! 30 160 find! 31 161 find! 32 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 find! 33 182 find! 34 183 find! 35 184 find! 36 185 find! 37 186 find! 38 187 fin

PicklingError: Can't pickle <type 'cv2.VideoCapture'>: it's not the same object as cv2.VideoCapture

In [4]:
# import cv2
# import numpy as np
# from collections import deque
# import pickle
# import dill

# filename = 'globalsave1.pkl'
# dill.load_session(filename)


In [4]:
delay = 150
if nFrame <= nFrame4calib:
    idxs = range(nFrame)
else:
    if bRandomSample:
        idxs = np.sort(np.random.choice(nFrame, nFrame4calib, replace=False)).tolist()
    else:
        idxs = np.linspace(0,nFrame-1,nFrame4calib).astype(int).tolist()
# idxs = range(0,len(valid_frames),3)

print len(idxs)
for idx in idxs:
    print "frame ",idx, 
    frame = valid_frames[idx]
    gray = valid_grays[idx]
    corners = valid_corners[idx]

    objpoints.append(objp)
    corners = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints.append(corners)
    cv2.drawChessboardCorners(frame, nCorners, corners, find)
    frames.append(frame) 
    if bShow:
        cv2.imshow('Frame',frame)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
print

cv2.destroyAllWindows()

cali_ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

print mtx
print dist

print len(idxs)
print len(rvecs)
print len(tvecs)

np.savetxt(outfile, np.append(mtx, dist))

# filename = 'globalsave2.pkl'
# dill.dump_session(filename)


100
frame  1 frame  2 frame  3 frame  4 frame  6 frame  9 frame  10 frame  12 frame  16 frame  17 frame  21 frame  24 frame  25 frame  26 frame  27 frame  28 frame  29 frame  30 frame  31 frame  32 frame  33 frame  37 frame  39 frame  40 frame  41 frame  42 frame  43 frame  46 frame  47 frame  48 frame  49 frame  50 frame  51 frame  53 frame  54 frame  55 frame  58 frame  60 frame  64 frame  66 frame  67 frame  68 frame  69 frame  71 frame  72 frame  74 frame  75 frame  76 frame  77 frame  78 frame  79 frame  80 frame  82 frame  84 frame  87 frame  88 frame  89 frame  90 frame  92 frame  93 frame  94 frame  96 frame  97 frame  98 frame  99 frame  102 frame  104 frame  107 frame  109 frame  111 frame  112 frame  114 frame  117 frame  119 frame  120 frame  121 frame  122 frame  127 frame  130 frame  131 frame  132 frame  133 frame  134 frame  135 frame  136 frame  139 frame  141 frame  142 frame  145 frame  146 frame  148 frame  150 frame  152 frame  153 frame  154 frame  155 frame  156 

NameError: name 'outfile' is not defined

In [5]:
np.savetxt('out.txt', np.append(mtx, dist))

In [12]:
# import cv2
# import numpy as np
# from collections import deque
# import pickle
# import dill

# filename = 'globalsave2.pkl'
# dill.load_session(filename)
# np.savetxt('out.txt', np.append(mtx, dist))

In [6]:
idxs_test = range(0,len(valid_frames),10)

######################### test ##########
objpoints_test = [] # 3d point in real world space
imgpoints_test = [] # 2d points in image plane.
frames_test = []
print len(idxs_test)
for idx in idxs_test:
    print "frame ",idx
    frame = valid_frames[idx]
    gray = valid_grays[idx]
    corners = valid_corners[idx]

    objpoints_test.append(objp)
    corners = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints_test.append(corners)
    cv2.drawChessboardCorners(frame, nCorners, corners, find)
    frames_test.append(frame) 
    # if bShow:
    #     cv2.imshow('Frame',frame)
    #     if cv2.waitKey(delay) & 0xFF == ord('q'):
    #         break

cv2.destroyAllWindows()


cali_ret, _, _, rvecs_test, tvecs_test = cv2.calibrateCamera(objpoints_test, imgpoints_test, 
                                                             gray.shape[::-1], mtx, dist)




17
frame  0
frame  10
frame  20
frame  30
frame  40
frame  50
frame  60
frame  70
frame  80
frame  90
frame  100
frame  110
frame  120
frame  130
frame  140
frame  150
frame  160


In [7]:
bShow = True
delay = 500


tot_error = 0
tot_error_undist = 0
mean_error = 0


def MyProjection(objp, rvec, tvec, mtx, dst):
    rot, _ = cv2.Rodrigues(rvec)
    imgp = np.dot(rot,objp.T) + tvec
    imgp = imgp / imgp[2,:]

    x = imgp[0,:]
    y = imgp[1,:]
    
    r = np.sqrt(x**2 + y**2)

    k1, k2, p1, p2, k3 = dst
    c = (1 + k1*r**2 + k2*r**4 + k3*r**6)/(1)
    x_new = c*x + 2*p1*x*y + p2*(r**2 + 2*x**2)
    y_new = c*y + p1*(r**2 + 2*y**2) + 2*p2*x*y

    imgp[0,:]= x_new
    imgp[1,:]= y_new
    

    imgp = np.dot(mtx,imgp)[:2,:].T.astype(objp.dtype)
    return imgp


# for frame, rvec, tvec, corners, objp in zip(frames_test, rvecs_test, tvecs_test, imgpoints_test, objpoints_test):
for i in range(len(frames_test)):
    frame, rvec, tvec, corners, objp = frames_test[i], rvecs_test[i], tvecs_test[i], imgpoints_test[i], objpoints_test[i]
    imgp1 = MyProjection(objp, rvec, tvec, mtx, dist[0])
    # imgpoints1, _ = cv2.projectPoints(objp, rvec, tvec, mtx, dist)
    # print cv2.norm(imgpoints1.reshape(imgp1.shape), imgp1, cv2.NORM_L2)
    
    for p in imgp1:
        cv2.circle(frame, tuple(p.astype(int)), 1, (0,0,255), 3)

    tot_error += cv2.norm(corners,imgp1.reshape(corners.shape), cv2.NORM_L2)/len(imgp1)

    if bShow:
        cv2.imshow('Frame',frame)
        # cv2.imshow('Dst',dst)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
           break

print "total error: ", tot_error/len(objpoints)








total error:  0.0291172612843
